# Fifa World Cup 2018 - Prediction

## Brainstorming

This section is dedicated to the Exploratory Data Analysis and brainstoriming on possible faetures and problem formulation.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlalchemy
%matplotlib inline

### Dataset 1: [International football results from 1872 to 2018](https://www.kaggle.com/martj42/international-football-results-from-1872-to-2017/data):

This data set contains the results of internation games for the past 146 years.

AG: Let's import the main data set(international games).
With this we can get some data for all the available games till now.
Especially the last 2 decades of these data can be used to extract some useful information for each team. These are the features I think we can get out of this data:

- Win to lose ratio total
- Win to lose ration general
- Mean goal scored ( **this can be used to define weather team is defensive or not** )
- Mean goal recieved ( **this can be used to define weather team is offensive or not** )

In [2]:
international_matches = pd.read_csv('international_matches.zip', compression='zip', header=0, sep=',', quotechar='"')

In [3]:
international_matches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38902 entries, 0 to 38901
Data columns (total 8 columns):
date          38902 non-null object
home_team     38902 non-null object
away_team     38902 non-null object
home_score    38902 non-null int64
away_score    38902 non-null int64
tournament    38902 non-null object
city          38902 non-null object
country       38902 non-null object
dtypes: int64(2), object(6)
memory usage: 1.5+ MB


 ### Initial content's dataset

In [4]:
international_matches.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland
1,1873-03-08,England,Scotland,4,2,Friendly,London,England
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland
3,1875-03-06,England,Scotland,2,2,Friendly,London,England
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland


In [4]:
all_series = {}

In [5]:
def generate_analytics_team(row):
    
    team = row.home_team
    
    victorie = row.home_score > row.away_score
    lose = row.home_score < row.away_score
    tie = row.home_score == row.away_score
    
    goals_scored = row.home_score
    goals_received = row.away_score
    
    if team not in all_series:
        serie_team = {"Team": team, "Victories": int(victorie), "Loses": int(lose), "Ties": int(tie), "Ratio(win/lose)": 0, "Goals(scored)": goals_scored, "Goals(received)": goals_received}
        all_series[team] = serie_team
    else:
        
        if victorie:
            all_series[team]['Victories'] += 1
        elif lose:
            all_series[team]['Loses'] += 1
        else:
            all_series[team]['Ties'] += 1
            
        all_series[team]['Goals(scored)'] += goals_scored
        all_series[team]['Goals(received)'] += goals_received

In [8]:
international_matches[(international_matches.home_team == 'England') | (international_matches.away_team == 'England')]

,date,home_team,away_team,home_score,away_score,tournament,city,country
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland
1,1873-03-08,England,Scotland,4,2,Friendly,London,England
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland
3,1875-03-06,England,Scotland,2,2,Friendly,London,England
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland
6,1877-03-03,England,Scotland,1,3,Friendly,London,England
8,1878-03-02,Scotland,England,7,2,Friendly,Glasgow,Scotland
10,1879-01-18,England,Wales,2,1,Friendly,London,England
11,1879-04-05,England,Scotland,5,4,Friendly,London,England
13,1880-03-13,Scotland,England,5,4,Friendly,Glasgow,Scotland


In [6]:
for i , row in international_matches.iterrows():
    generate_analytics_team(row)

In [7]:
columns = ["Team","Victories","Loses","Ties","Ratio(win/lose)", "Goals(scored)","Goals(received)"]

infos_teams = pd.DataFrame(list(all_series.values()),columns=columns)
infos_teams.head()

,Team,Victories,Loses,Ties,Ratio(win/lose),Goals(scored),Goals(received)
0,Scotland,208,82,80,0,710,363
1,England,292,77,106,0,1087,426
2,Wales,117,118,78,0,456,444
3,Northern Ireland,103,135,67,0,392,521
4,USA,190,100,91,0,622,425


### Dataset 2: fifa 2018 schedule

> This dataset consist of all the games in the upcoming matches. What we are thinking of getting from this data is the name of the participating teams. Moreover, eventually we use them to predict all of the games and the champion.

In [29]:
fifa2018 = pd.read_csv('fifa2018.csv')

In [30]:
counteris2018 = fifa2018['Home Team'].unique()[:32]

In [31]:
counteris2018

array(['Russia', 'Egypt', 'Morocco', 'Portugal', 'France', 'Argentina',
       'Peru', 'Croatia', 'Costa Rica', 'Germany', 'Brazil', 'Sweden',
       'Belgium', 'Tunisia', 'Colombia', 'Poland', 'Uruguay', 'Iran',
       'Denmark', 'Nigeria', 'Serbia', 'Korea Republic', 'England',
       'Japan', 'Saudi Arabia', 'Spain', 'Australia', 'Iceland', 'Mexico',
       'Switzerland', 'Senegal', 'Panama'], dtype=object)

### Dataset 3: [European Soccer Database](https://www.kaggle.com/hugomathien/soccer/data):

(can anybody add some explanation here?)
!!! You need to extract the players.sqlite.zip to the same folder !!!

In [46]:
import sqlite3
uri_database = "sqlite:///database.sqlite"

In [47]:
players = pd.read_sql_table("Player",uri_database)
players_attributes = pd.read_sql_table("Player_Attributes",uri_database)
team_attributes = pd.read_sql_table("Team_Attributes", uri_database)

In [50]:
print("Players DF: ", players.shape)
print("Players attributes DF: ", players_attributes.shape)
print("Team attributes DF: ", team_attributes.shape)

Players DF:  (11060, 7)
Players attributes DF:  (183978, 42)
Team attributes DF:  (1458, 25)


## EDA
Can someone do some EDA?